# Генерация праздничных тостов 
Для работы с GPT скачаем предобученную модель.
Мы будем работать с библиотекой `transformers` от Hugging Face и с русскоязычной моделью ruGPT3 от Сбера.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [59]:
!pip install transformers 

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


loading file vocab.json from cache at /root/.cache/huggingface/hub/models--sberbank-ai--rugpt3small_based_on_gpt2/snapshots/f2f7c585b05a16726efe8974586e10b4d5939082/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--sberbank-ai--rugpt3small_based_on_gpt2/snapshots/f2f7c585b05a16726efe8974586e10b4d5939082/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--sberbank-ai--rugpt3small_based_on_gpt2/snapshots/f2f7c585b05a16726efe8974586e10b4d5939082/config.json
Model config GPT2Config {
  "_name_or_path": "sberbank-ai/rugpt3small_based_on_gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false

## Обучающие данные
Будем учить GPT генерировать праздничные тосты. Спарсим сайт https://pozdravok.com/toast/ и выведем полученные данные в dataset.txt

In [7]:
!cp /content/drive/MyDrive/dataset/dataset_with_ST_ET.txt .

In [8]:
f=open("dataset_with_ST_ET.txt", 'r')
text = f.read()

In [12]:
import re

result = len(re.findall(r'\w+', text))
result1 = len(re.findall(r'\w', text))

print("There are " + str(result) + " words.")
print("There are " + str(result1) + " chars.")

There are 55194 words.
There are 284070 chars.


In [13]:
from functools import reduce
reduce( lambda d, c: d.update([(c, d.get(c,0)+1)]) or d, text.split(), {}) 

{'[ST]От': 21,
 'всей': 61,
 'души': 58,
 'поздравляю': 42,
 'с': 768,
 'праздником!': 23,
 'Желаю': 436,
 'безмерного': 4,
 'счастья,': 114,
 'удачи,': 37,
 'непоколебимого': 2,
 'здоровья,': 116,
 'побольше': 52,
 'белых': 1,
 'полос': 4,
 'по': 127,
 'жизни': 197,
 'и': 3253,
 'самых': 49,
 'добрых,': 2,
 'искренних': 10,
 'чудесных': 10,
 'моментов![ET]': 3,
 '[ST]В': 67,
 'этот': 240,
 'прекрасный': 28,
 'день': 262,
 'хочется': 42,
 'пожелать': 140,
 'успехов,': 23,
 'благополучия,': 30,
 'творческих': 5,
 'побед': 10,
 'увлекательных': 4,
 'приключений.': 3,
 'Пусть': 820,
 'все,': 21,
 'что': 272,
 'вы': 41,
 'наметили': 1,
 'сбудется': 8,
 'как': 236,
 'можно': 51,
 'скорее.': 2,
 'Жизнь': 8,
 'прекрасна': 2,
 '—': 309,
 'наслаждайтесь': 2,
 'каждым': 29,
 'ее': 22,
 'мигом![ET]': 1,
 '[ST]Поздравляю': 259,
 'От': 16,
 'желаю,': 40,
 'чтобы': 549,
 'жизнь': 211,
 'протекала': 1,
 'без': 67,
 'проблем,': 6,
 'зарядом': 1,
 'позитива,': 15,
 'добрыми': 10,
 'людьми': 6,
 'рядом'

In [14]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import AutoTokenizer
from transformers import LineByLineTextDataset

train_path = 'dataset_with_ST_ET.txt'

#tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path, bos_token='[ST]', eos_token='[ET]', pad_token='[PAD]')

train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Creating features from dataset file at 
Saving features into cached file cached_lm_GPT2Tokenizer_64_dataset_with_ST_ET.txt [took 0.003 s]


## Training
Для файнтюнинга нам понадобится объект класса Trainer. Далее нужно будет запустить `trainer.train()`

In [61]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned", 
    overwrite_output_dir=True,
    num_train_epochs=200,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=10,
    gradient_accumulation_steps=16
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None)
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [62]:
trainer.train()

***** Running training *****
  Num examples = 1310
  Num Epochs = 200
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 400
  Number of trainable parameters = 125231616


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=400, training_loss=2.7929037475585936, metrics={'train_runtime': 4060.8557, 'train_samples_per_second': 64.518, 'train_steps_per_second': 0.099, 'total_flos': 8547972857856000.0, 'train_loss': 2.7929037475585936, 'epoch': 199.78})

# Результат файнтюнинга
Будем использовать сэмплирование с ограничением

In [ ]:
!cp /content/drive/MyDrive/model_best_200.hdf5 .

In [ ]:
!ls

cached_lm_GPT2Tokenizer_64_dataset.txt	     drive		  sample_data
cached_lm_GPT2Tokenizer_64_dataset.txt.lock  finetuned
dataset.txt				     model_best_500.hdf5


In [81]:
PATH = '/content/drive/MyDrive/model_best_200_ST.hdf5'
#torch.save(model, PATH)

#trainer.save_model("/content/drive/MyDrive")
#model.save_pretrained("/content/drive/MyDrive")

#the_model = torch.load("model_best_200.hdf5")
model.eval()

text = "С Новым Годом!\n"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)

with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5,
                        temperature=1.8,
                        top_p=0.8,
                        top_k=50,
                        max_length=120,
                        min_length=30,
                        no_repeat_ngram_size=2 
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)
print()
t=0
for s in generated_text:
  if s=='[' or s=='E' or s=='T' or s==']':
    t=t+1
  elif (s!='S'):
    print(s, end="")
  if t==4:
    break
print()
#print(generated_text)

Saving model checkpoint to /content/drive/MyDrive
Configuration saved in /content/drive/MyDrive/config.json
Model weights saved in /content/drive/MyDrive/pytorch_model.bin
Configuration saved in /content/drive/MyDrive/config.json
Model weights saved in /content/drive/MyDrive/pytorch_model.bin
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



С Новым Годом!
[ST]Пусть этот год будет самым счастливым и самым успешным в твоей жизни. Пусть все плохое и ненастья останутся в уходящем году, а впереди тебя ждет лишь светлое и радостное будущее! Желаю тебе всего наилучшего в Новом Году и пусть в нём тебя ждут только приятные сюрпризы и приятные встречи. Желаю, чтобы в этом году тебя окружали только верные и любящие люди, пусть каждый день будет наполнен только позитивом и счастьем! С Новым годом, с новым счастьем и новыми успехами! Пусть этот Новый год принесёт тебе только радость и счастье!

С Новым Годом!
Пусть этот год будет самым счастливым и самым успешным в твоей жизни. Пусть все плохое и ненастья останутся в уходящем году, а впереди тебя ждет лишь светлое и радостное будущее! Желаю тебе всего наилучшего в Новом Году и пусть в нём тебя ждут только приятные сюрпризы и приятные встречи. Желаю, чтобы в этом году тебя окружали только верные и любящие люди, пусть каждый день будет наполнен только позитивом и счастьем! С Новым годо